# operazioni preliminari

In [ ]:
import os
import numpy as np
import os
from PIL import Image, ImageDraw

from torch.utils.data import DataLoader

import torch, torchvision
import torchvision.transforms as T
import pycocotools.coco as coco
from pycocotools.coco import COCO
import skimage.io as io
import matplotlib.pyplot as plt
import pylab

!pip install wget

In [ ]:
print(torch.__version__, torch.cuda.is_available())
torch.set_grad_enabled(False);

In [ ]:
from torchvision.transforms.transforms import Grayscale
# standard PyTorch mean-std input image normalization
transform = T.Compose([
    T.Resize(800),
    T.ToTensor(),
    T.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

# for output bounding box post-processing
def box_cxcywh_to_xyxy(x):
    x_c, y_c, w, h = x.unbind(1)
    b = [(x_c - 0.5 * w), (y_c - 0.5 * h),
         (x_c + 0.5 * w), (y_c + 0.5 * h)]
    return torch.stack(b, dim=1)

def rescale_bboxes(out_bbox, size):
    img_w, img_h = size
    b = box_cxcywh_to_xyxy(out_bbox)
    b = b * torch.tensor([img_w, img_h, img_w, img_h], dtype=torch.float32)
    return b

In [ ]:
#clone woctezuma detr
%cd /content/

!rm -rf detr
!git clone https://github.com/woctezuma/detr.git

%cd detr/

!git checkout finetune

In [ ]:
#carica pesi pre-addestrati
# Get pretrained weights
checkpoint = torch.hub.load_state_dict_from_url(
            url='https://dl.fbaipublicfiles.com/detr/detr-r50-e632da11.pth',
            map_location='cpu',
            check_hash=True)

# Remove class weights
del checkpoint["model"]["class_embed.weight"]
del checkpoint["model"]["class_embed.bias"]

# Save
torch.save(checkpoint,
           'detr-r50_no-class-head.pth')

# operazioni su dataset per fine-tuning

In [ ]:
path_ds = '/content/path_to_dataset'

In [ ]:
path_annotation_train = os.path.join(path_ds,'annotations/custom_train.json')

controllo su dataset

In [ ]:
# initialize COCO api for instance annotations
coco=COCO(path_annotation_train)

In [ ]:
# display COCO categories and supercategories
cats = coco.loadCats(coco.getCatIds())

nms=[cat['name'] for cat in cats]
print('Categories: {}'.format(nms))

nms = set([cat['supercategory'] for cat in cats])
print('Super-categories: {}'.format(nms))

In [ ]:
# load and display image
catIds = coco.getCatIds(catNms=['nome_categoria_da_ispezionare']);
imgIds = coco.getImgIds(catIds=catIds );

In [ ]:
pylab.rcParams['figure.figsize'] = (8.0, 8.0)

In [ ]:
img_id = imgIds[np.random.randint(0,len(imgIds))]
print('Image n°{}'.format(img_id))

img = coco.loadImgs(img_id)[0]

img_name = '%s/%s/%s'%(path_detr, '/train2017', img['file_name'])
print('Image name: {}'.format(img_name))

I = io.imread(img_name)
plt.figure()
plt.imshow(I)

In [ ]:
annIds = coco.getAnnIds(imgIds=img['id'], catIds=catIds)
anns = coco.loadAnns(annIds)

In [ ]:
# load and display instance annotations
plt.imshow(I)
coco.showAnns(anns, draw_bbox=True)

# fine-tuning

In [ ]:
num_classes = numero_di_categorie

CLASSES = finetuned_classes = [
      'N/A',
      'categoria_i-esima',
]

In [ ]:
#training loop
!python main.py \
  --dataset_file "custom" \
  --coco_path "/content/path_to_coco_ds" \
  --output_dir "/content/output_directory" \
  --resume "detr-r50_no-class-head.pth" \
  --num_classes $num_classes \
  --epochs 10

In [ ]:
torch.save(model.state_dict(), '/content/path_to_model/.pth')

In [ ]:
#load fine tuned model
model = torch.hub.load('facebookresearch/detr',
                       'detr_resnet50',
                       pretrained=False,
                       num_classes=num_classes)

checkpoint = torch.load('/content/drive/MyDrive/.../checkpoint.pth',
                        map_location='cpu')

model.load_state_dict(checkpoint['model'],
                      strict=False)

model.eval();

# inferenza con modello fine tuned

In [ ]:
# colors for visualization
COLORS = [[0.000, 0.447, 0.741], [0.850, 0.325, 0.098], [0.929, 0.694, 0.125],
          [0.494, 0.184, 0.556], [0.466, 0.674, 0.188], [0.301, 0.745, 0.933]]

In [ ]:
def plot_finetuned_results(pil_img, prob=None, boxes=None):
    plt.figure(figsize=(16,10))
    plt.imshow(pil_img)
    ax = plt.gca()
    colors = COLORS * 100
    if prob is not None and boxes is not None:
      for p, (xmin, ymin, xmax, ymax), c in zip(prob, boxes.tolist(), colors):
          ax.add_patch(plt.Rectangle((xmin, ymin), xmax - xmin, ymax - ymin,
                                    fill=False, color=c, linewidth=3))
          cl = p.argmax()
          text = f'{finetuned_classes[cl]}: {p[cl]:0.2f}'
          ax.text(xmin+5, ymin-15, text, fontsize=15,
                  bbox=dict(facecolor='yellow', alpha=0.5))
    plt.axis('off')
    plt.savefig('/content/inferenza1.jpg',transparent = True, bbox_inches = 'tight', pad_inches = 0)

In [ ]:
def filter_bboxes_from_outputs(outputs,
                               threshold=0.7):
  
  # keep only predictions with confidence above threshold
  probas = outputs['pred_logits'].softmax(-1)[0, :, :-1]
  keep = probas.max(-1).values > threshold

  probas_to_keep = probas[keep]

  # convert boxes from [0; 1] to image scales
  bboxes_scaled = rescale_bboxes(outputs['pred_boxes'][0, keep], im.size)
  
  return probas_to_keep, bboxes_scaled

In [ ]:

def inference_single(my_image, my_model):
  # mean-std normalize the input image (batch-size: 1)
  img = transform(my_image).unsqueeze(0)

  # propagate through the model
  outputs = my_model(img)

  for threshold in [0.9, 0.7]:
    probas_to_keep, bboxes_scaled = filter_bboxes_from_outputs(outputs,
                                                              threshold=threshold)

    plot_finetuned_results(my_image,
                           probas_to_keep, 
                           bboxes_scaled)


In [ ]:
#inferenza su immagine singola

img_name = '/content/path_to_image/.jpg'
im = Image.open(img_name)
im = im.convert('RGB')
inference_single((im), model)

In [ ]:
#inferenza su cartella immagini
import glob

for filename in glob.glob('/content/drive/MyDrive/.../*.jpg'): 
    nome = (os.path.basename(filename))
    im=Image.open(filename)

    img = transform(im).unsqueeze(0)

    # propagate through the model
    outputs = model(img)

    # keep only predictions with 0.7+ confidence
    probas = outputs['pred_logits'].softmax(-1)[0, :, :-1]
    keep = probas.max(-1).values > 0.9

    # convert boxes from [0; 1] to image scales
    bboxes_scaled = rescale_bboxes(outputs['pred_boxes'][0, keep], im.size)

    plt.figure(figsize=(16,10))
    plt.imshow(im)
    ax = plt.gca()
    colors = COLORS * 100
    for p, (xmin, ymin, xmax, ymax), c in zip(probas[keep], bboxes_scaled.tolist(), colors):
          ax.add_patch(plt.Rectangle((xmin, ymin), xmax - xmin, ymax - ymin,
                                   fill=False, color=c, linewidth=3))
          cl = p.argmax()
          text = f'{CLASSES[cl]}: {p[cl]:0.2f}'
          ax.text(xmin, ymin, text, fontsize=15,
                bbox=dict(facecolor='yellow', alpha=0.5))
    plt.axis('off')
    path = os.path.join("/content/",nome)
    plt.savefig(path,transparent = True, bbox_inches = 'tight', pad_inches = 0)
    plt.close()
    print(nome)